# Electoral College Data ETL

The purpose of this project is to analyze the how the electoral college votes were distributed after the 2010 census, look at how the redistribution of population (by estimate) has shifted over time until today, and what that means for the voter per electoral college vote in each state. I will also analyze how we expect the electoral college to be redistributed after the 2020 census, given census bureau predictions.

An analysis will also be performed on the percentage likelihood of each state giving its electoral college votes to a particular party and their respective nominees, based purely on historical data. The aim is to demonstrate which states have the greatest power per vote, given both their current electoral votes allotted and the likelihood of that state assigning thier votes to either candidate.

## Import

In [1]:
#import necessary libraries

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

import requests
import json
import copy

from bs4 import BeautifulSoup
import re

import sqlite3
%matplotlib inline

## Support Functions

In [2]:
#define how to retrieve api keys

def get_keys(path):
    """
    Pulls necessary api keys from designated path
    """
    with open(path) as f:
        return json.load(f)

In [3]:
def reciprocal_geometric_mean(next_house_seat):
    """
    Calculates the reciprocal geometric mean for the next house seat a state could potentially receive
    """
    return 1 / np.sqrt(next_house_seat*(next_house_seat-1))

In [4]:
def priority_value(state_pop, next_house_seat):
    """
    Calculates the priority value a state has for receiving another house seat.
    The highest priority value state gets the next house seat available.
    """
    return int(round(state_pop * reciprocal_geometric_mean(next_house_seat), 0))

In [5]:
def state_status_scrape(state_start):
    """
    From a beautiful soup webscrape, takes the tag just prior to the first state with NPVIC information
    and iterates through all siblings to make and return a dictionary of all states and their NPVIC status
    """
    state_npvic_status = {}
    for p in state_start.find_next_siblings():
        if p.contents == [] or p.contents == ['\xa0']:
            break

        soup_state_status = p.get_text().replace('\xa0', ' ')
        if '-' not in soup_state_status:
            state = soup_state_status
            status = ''
        else:
            dash_index = soup_state_status.index('-')
            state = soup_state_status[:dash_index-1]
            status = soup_state_status[dash_index+2:]

        state_npvic_status.update({state: status})
        
    return state_npvic_status

In [6]:
def state_status_to_numerical(state_npvic_status):
    """
    Takes all phrases from dictionary and assigns a value for how close the state is to joining the NPVIC
    Joined (passed legislature and signed by governor): 1.0
    Passed by one or two branches of state legislature: 0.5
    Approved by house committee:                        0.25
    States with no movement:                            0
    """
    for state in state_npvic_status.keys():
        if 'Enacted into law' in state_npvic_status[state]:
            state_npvic_status[state] = 1.0
        elif 'Passed' in state_npvic_status[state]:
            state_npvic_status[state] = 0.5
        elif 'approved' in state_npvic_status[state]:
            state_npvic_status[state] = 0.25
        else:
            state_npvic_status[state] = 0

    return state_npvic_status

In [7]:
def sql_table_creation(connection, cursor, table_name, *argv):
    var_string = argv[0]
    var_list = argv[0].split(' ')[0]
    for arg in argv[1:]:
        var_string += ', ' + arg
        var_list += ', ' + arg.split(' ')[0]
        
    try:
        c.execute('CREATE TABLE {} ({})'.format(table_name, var_string))
        conn.commit()
        print('{} Table created ({})'.format(table_name, var_list))
    except:
        c.execute('DROP TABLE {}'.format(table_name))
        print('{} table dropped'.format(table_name))
        c.execute('CREATE TABLE {} ({})'.format(table_name, var_string))
        conn.commit()
        print('{} table created ({})'.format(table_name, var_list))

## Data Collection

### API Request

In [8]:
#get key for census bureau api
key_path = "/Users/flatironschool/.secret/census_api.json"
keys = get_keys(key_path)

api_key = keys['api_key']

In [9]:
#make and print request for census count and estimates from 2010-2019 for all states
year = '2019'

url = 'https://api.census.gov/data/{}/pep/population'.format(year)

variables = ['DATE_CODE', 'DATE_DESC', 'POP', 'NAME']

granularity = 'state:*'

params = {'get': ','.join(variables), 'for': granularity, 'key': api_key}

r = requests.get(url, params=params)
print(r.url)
print(r)
print(r.text[:1000])

https://api.census.gov/data/2019/pep/population?get=DATE_CODE%2CDATE_DESC%2CPOP%2CNAME&for=state%3A%2A&key=b7961d22ec04ff1777be8a0450921d3f28af8315
<Response [200]>
[["DATE_CODE","DATE_DESC","POP","NAME","state"],
["1","4/1/2010 Census population","5303925","Minnesota","27"],
["2","4/1/2010 population estimates base","5303927","Minnesota","27"],
["3","7/1/2010 population estimate","5310828","Minnesota","27"],
["4","7/1/2011 population estimate","5346143","Minnesota","27"],
["5","7/1/2012 population estimate","5376643","Minnesota","27"],
["6","7/1/2013 population estimate","5413479","Minnesota","27"],
["7","7/1/2014 population estimate","5451079","Minnesota","27"],
["8","7/1/2015 population estimate","5482032","Minnesota","27"],
["9","7/1/2016 population estimate","5522744","Minnesota","27"],
["10","7/1/2017 population estimate","5566230","Minnesota","27"],
["11","7/1/2018 population estimate","5606249","Minnesota","27"],
["12","7/1/2019 population estimate","5639632","Minnesota","27"],

In [11]:
#take the api response, turn it into a json, then parse that json into a pandas dataframe
data = r.json()
pop_df = pd.DataFrame(data[1:], columns=data[0])

#cut down the date column to just the year
pop_df['YEAR'] = pop_df.DATE_DESC.apply(lambda x: x[4:8])

#eliminate the census population estimate for the same year it was recorded and the July date of that same year
pop_df.drop(pop_df[(pop_df.DATE_CODE == '2') | (pop_df.DATE_CODE == '3')].index, inplace=True)

#drop unnecessary columns and reset the index
pop_df.drop(['state','DATE_CODE', 'DATE_DESC'], axis=1, inplace=True)
pop_df.reset_index(drop=True, inplace=True)

#set the population to an integer val, reorder and rename the columns
pop_df['POP'] = pop_df.POP.astype('int64')
pop_df = pop_df[['YEAR', 'NAME', 'POP']]
pop_df.columns = ['Year', 'State', 'Population']

display(pop_df.head(15))
display(pop_df.info())

,Year,State,Population
0,2010,Minnesota,5303925
1,2011,Minnesota,5346143
2,2012,Minnesota,5376643
3,2013,Minnesota,5413479
4,2014,Minnesota,5451079
5,2015,Minnesota,5482032
6,2016,Minnesota,5522744
7,2017,Minnesota,5566230
8,2018,Minnesota,5606249
9,2019,Minnesota,5639632


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 520 entries, 0 to 519
Data columns (total 3 columns):
Year          520 non-null object
State         520 non-null object
Population    520 non-null int64
dtypes: int64(1), object(2)
memory usage: 12.3+ KB


None

### Overseas Population Import

In [12]:
overseas_df = pd.read_excel('Overseas Population 2010.xls', skiprows=7)
overseas_df.dropna(inplace=True)
overseas_df.columns = ['State', 'Overseas_pop']
overseas_df['Year'] = '2010'
overseas_df['Overseas_pop'] = overseas_df['Overseas_pop'].astype('int64')
overseas_df.reset_index(drop=True, inplace=True)

display(overseas_df.head())
display(overseas_df.info())

,State,Overseas_pop,Year
0,Alabama,23246,2010
1,Alaska,11292,2010
2,Arizona,20683,2010
3,Arkansas,10311,2010
4,California,88033,2010


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 52 entries, 0 to 51
Data columns (total 3 columns):
State           52 non-null object
Overseas_pop    52 non-null int64
Year            52 non-null object
dtypes: int64(1), object(2)
memory usage: 1.3+ KB


None

In [13]:
#merge the overseas data with the main population estimates table and extrapolate overseas estimates based on 
#percentage change in the population year over year

#merge the population estimates with overseas data on State and Year
merged_df = pop_df.merge(overseas_df, how='left', on=['State','Year'])

#create our Percent change in population column
merged_df['Percent_change'] = (merged_df.Population - merged_df.Population.shift(1)) / merged_df.Population.shift(1)

#iterate through the missing data in overseas population, calculating the next missing value from the previous known
#(from the 2010 census) or from the last calculated value based on percentage change in that state
for i in range(len(merged_df)):
    if pd.isna(merged_df.iloc[i]['Overseas_pop']):
        merged_df.at[i,'Overseas_pop'] = round(merged_df.iloc[i]['Percent_change'] * \
                                               merged_df.iloc[i-1]['Overseas_pop'] + \
                                               merged_df.iloc[i-1]['Overseas_pop'], 0)
        
merged_df.drop('Percent_change', axis=1, inplace=True)        
merged_df['Overseas_pop'] = merged_df.Overseas_pop.astype('int64')
merged_df['Year'] = merged_df.Year.astype('int64')

display(merged_df.head(25))
display(merged_df.info())

,Year,State,Population,Overseas_pop
0,2010,Minnesota,5303925,10954
1,2011,Minnesota,5346143,11041
2,2012,Minnesota,5376643,11104
3,2013,Minnesota,5413479,11180
4,2014,Minnesota,5451079,11258
5,2015,Minnesota,5482032,11322
6,2016,Minnesota,5522744,11406
7,2017,Minnesota,5566230,11496
8,2018,Minnesota,5606249,11579
9,2019,Minnesota,5639632,11648


<class 'pandas.core.frame.DataFrame'>
Int64Index: 520 entries, 0 to 519
Data columns (total 4 columns):
Year            520 non-null int64
State           520 non-null object
Population      520 non-null int64
Overseas_pop    520 non-null int64
dtypes: int64(3), object(1)
memory usage: 40.3+ KB


None

### Scraping National Popular Vote Interstate Compact State Status

In [14]:
url = "https://www.nationalpopularvote.com/state-status"
req = requests.get(url)
soup = BeautifulSoup(req.content, 'html.parser')

In [15]:
#skips over the initial list (and links) of states at the top of the page
init_state_skip = soup.find("p", text=re.compile("On the map below"))

In [16]:
#brings us to the tag just before the states and their current NPVIC status
tag_before_states = init_state_skip.find_next_sibling().find_next_sibling()

print(tag_before_states)
print(tag_before_states.find_next_sibling())

<p> </p>
<p><a class="menu__link" href="/state/ak">Alaska</a></p>


In [17]:
#scrape the site for NPVIC status of each state
npvic_word_status = state_status_scrape(tag_before_states)
#make the status numerical
npvic_num_status = state_status_to_numerical(npvic_word_status)

#turns dict into pandas dataframe
npvic_df = pd.DataFrame.from_dict(npvic_num_status, orient='index')
npvic_df.reset_index(inplace=True)
npvic_df.columns = ['State', 'NPVIC_num']

display(npvic_df.head())
display(npvic_df.info())

,State,NPVIC_num
0,Alaska,0.0
1,Alabama,0.0
2,Arkansas,0.5
3,Arizona,0.5
4,California,1.0


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 51 entries, 0 to 50
Data columns (total 2 columns):
State        51 non-null object
NPVIC_num    51 non-null float64
dtypes: float64(1), object(1)
memory usage: 944.0+ bytes


None

### Election History Data

Some things of note:
- In exploration of the data, there was a "democrat/republican" that someone once ran as (James Gritz, only in CO, and in 1992) but that was excluded from the purely democrat/republican divide I want to study here
- Any write in candidates were also not considered, as they are effectively outliers for this study
- This dataset was only for US Presidential elections, so the office column is superfluous
- In 2000, 2004, 2012, nominees Al Gore, John Kerry and Barack Obama had to run under the 'democratic-farmer-labor' party. For analysis, that party name has been changed to 'democrat' for ease of analysis

In [18]:
vote_history = pd.read_csv('1976-2016-president.csv')

#changing the democratic-farmer-labor party to just democrat for 2000, 2004, and 2012
vote_history.replace(to_replace='democratic-farmer-labor', value='democrat', inplace=True)

#drop anything that's a write in, or not in the main two parties, and reset the index
vote_history = vote_history[(vote_history['party'].isin(['democrat', 'republican'])) & 
                            (vote_history['writein'] == False)]
vote_history.reset_index(drop=True, inplace=True)

#get rid of unnecessary columns
drop_vote_history_cols = ['state_po', 'state_fips', 'state_cen', 'writein', 'state_ic', 'office', 'version', 'notes']
vote_history.drop(drop_vote_history_cols, axis=1, inplace=True)

display(vote_history.head(10))
display(vote_history.info())

,year,state,candidate,party,candidatevotes,totalvotes
0,1976,Alabama,"Carter, Jimmy",democrat,659170,1182850
1,1976,Alabama,"Ford, Gerald",republican,504070,1182850
2,1976,Alaska,"Ford, Gerald",republican,71555,123574
3,1976,Alaska,"Carter, Jimmy",democrat,44058,123574
4,1976,Arizona,"Ford, Gerald",republican,418642,742719
5,1976,Arizona,"Carter, Jimmy",democrat,295602,742719
6,1976,Arkansas,"Carter, Jimmy",democrat,498604,767535
7,1976,Arkansas,"Ford, Gerald",republican,267903,767535
8,1976,California,"Ford, Gerald",republican,3882244,7803770
9,1976,California,"Carter, Jimmy",democrat,3742284,7803770


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1122 entries, 0 to 1121
Data columns (total 6 columns):
year              1122 non-null int64
state             1122 non-null object
candidate         1122 non-null object
party             1122 non-null object
candidatevotes    1122 non-null int64
totalvotes        1122 non-null int64
dtypes: int64(3), object(3)
memory usage: 52.7+ KB


None

### Merging all data into SQL Database

In [19]:
#put all data into SQL database for retrieval in analysis
conn = sqlite3.connect('census_pop_data.db')
c = conn.cursor()

#### Population Data

In [20]:
sql_table_creation(conn, c, 'Population', 'Year integer', 'State text', 'Pop integer', 'Overseas_pop integer')

Population Table created (Year, State, Pop, Overseas_pop)


In [21]:
merged_df.to_sql(name='Population', con=conn, if_exists='replace')

#### NPVIC Status

In [22]:
sql_table_creation(conn, c, 'NPVIC_status', 'State text', 'Status real')

NPVIC_status Table created (State, Status)


In [23]:
npvic_df.to_sql(name='NPVIC_status', con=conn, if_exists='replace')

#### Election Results

In [26]:
sql_table_creation(conn, c, 'Election_results', 'Year integer', 'State text', 'Candidate text', 
                    'Party text', 'Candidatevotes integer', 'Totalvotes integer')

Election_results table dropped
Election_results table created (Year, State, Candidate, Party, Candidatevotes, Totalvotes)


In [27]:
vote_history.to_sql(name='Election_results', con=conn, if_exists='replace')

### Check the input

In [28]:
c.execute('SELECT * FROM population')
for row in c.fetchall():
    print(row)

(0, 2010, 'Minnesota', 5303925, 10954)
(1, 2011, 'Minnesota', 5346143, 11041)
(2, 2012, 'Minnesota', 5376643, 11104)
(3, 2013, 'Minnesota', 5413479, 11180)
(4, 2014, 'Minnesota', 5451079, 11258)
(5, 2015, 'Minnesota', 5482032, 11322)
(6, 2016, 'Minnesota', 5522744, 11406)
(7, 2017, 'Minnesota', 5566230, 11496)
(8, 2018, 'Minnesota', 5606249, 11579)
(9, 2019, 'Minnesota', 5639632, 11648)
(10, 2010, 'Mississippi', 2967297, 10943)
(11, 2011, 'Mississippi', 2978731, 10985)
(12, 2012, 'Mississippi', 2983816, 11004)
(13, 2013, 'Mississippi', 2988711, 11022)
(14, 2014, 'Mississippi', 2990468, 11028)
(15, 2015, 'Mississippi', 2988471, 11021)
(16, 2016, 'Mississippi', 2987938, 11019)
(17, 2017, 'Mississippi', 2988510, 11021)
(18, 2018, 'Mississippi', 2981020, 10993)
(19, 2019, 'Mississippi', 2976149, 10975)
(20, 2010, 'Missouri', 5988927, 22551)
(21, 2011, 'Missouri', 6010275, 22631)
(22, 2012, 'Missouri', 6024367, 22684)
(23, 2013, 'Missouri', 6040715, 22746)
(24, 2014, 'Missouri', 6056202, 22

In [29]:
c.execute('SELECT * FROM npvic_status')
for row in c.fetchall():
    print(row)

(0, 'Alaska', 0.0)
(1, 'Alabama', 0.0)
(2, 'Arkansas', 0.5)
(3, 'Arizona', 0.5)
(4, 'California', 1.0)
(5, 'Colorado', 1.0)
(6, 'Connecticut', 1.0)
(7, 'District of Columbia', 1.0)
(8, 'Delaware', 1.0)
(9, 'Florida', 0.0)
(10, 'Georgia', 0.25)
(11, 'Hawaii', 1.0)
(12, 'Iowa', 0.0)
(13, 'Idaho', 0.0)
(14, 'Illinois', 1.0)
(15, 'Indiana', 0.0)
(16, 'Kansas', 0.0)
(17, 'Kentucky', 0.0)
(18, 'Louisiana', 0.0)
(19, 'Massachusetts', 1.0)
(20, 'Maryland', 1.0)
(21, 'Maine', 0.5)
(22, 'Michigan', 0.5)
(23, 'Minnesota', 0.5)
(24, 'Missouri', 0.25)
(25, 'Mississippi', 0.0)
(26, 'Montana', 0.0)
(27, 'North Carolina', 0.5)
(28, 'North Dakota', 0.0)
(29, 'Nebraska', 0.0)
(30, 'New Hampshire', 0.0)
(31, 'New Jersey', 1.0)
(32, 'New Mexico', 1.0)
(33, 'Nevada', 0.5)
(34, 'New York', 1.0)
(35, 'Ohio', 0.0)
(36, 'Oklahoma', 0.5)
(37, 'Oregon', 1.0)
(38, 'Pennsylvania', 0.0)
(39, 'Rhode Island', 1.0)
(40, 'South Carolina', 0.0)
(41, 'South Dakota', 0.0)
(42, 'Tennessee', 0.0)
(43, 'Texas', 0.0)
(44, 'Ut

In [30]:
c.execute('SELECT * FROM election_results')
for row in c.fetchall():
    print(row)

(0, 1976, 'Alabama', 'Carter, Jimmy', 'democrat', 659170, 1182850)
(1, 1976, 'Alabama', 'Ford, Gerald', 'republican', 504070, 1182850)
(2, 1976, 'Alaska', 'Ford, Gerald', 'republican', 71555, 123574)
(3, 1976, 'Alaska', 'Carter, Jimmy', 'democrat', 44058, 123574)
(4, 1976, 'Arizona', 'Ford, Gerald', 'republican', 418642, 742719)
(5, 1976, 'Arizona', 'Carter, Jimmy', 'democrat', 295602, 742719)
(6, 1976, 'Arkansas', 'Carter, Jimmy', 'democrat', 498604, 767535)
(7, 1976, 'Arkansas', 'Ford, Gerald', 'republican', 267903, 767535)
(8, 1976, 'California', 'Ford, Gerald', 'republican', 3882244, 7803770)
(9, 1976, 'California', 'Carter, Jimmy', 'democrat', 3742284, 7803770)
(10, 1976, 'Colorado', 'Ford, Gerald', 'republican', 584278, 1081440)
(11, 1976, 'Colorado', 'Carter, Jimmy', 'democrat', 460801, 1081440)
(12, 1976, 'Connecticut', 'Ford, Gerald', 'republican', 719261, 1386355)
(13, 1976, 'Connecticut', 'Carter, Jimmy', 'democrat', 647895, 1386355)
(14, 1976, 'Delaware', 'Carter, Jimmy', '

In [31]:
c.close()
conn.close()